
## 1. 모듈 임포트
- tkinter 모듈은 파이썬에서 GUI를 만들기 위해 사용됩니다.
- ttk는 tkinter의 확장 모듈로, 스타일이 개선된 위젯을 제공합니다.
- random 모듈은 무작위 숫자 또는 데이터를 생성하는 데 사용됩니다.

In [1]:
import tkinter as tk  # 기본 tkinter 모듈
from tkinter import ttk  # 확장된 tkinter 위젯 모듈
import random  # 무작위 데이터 생성을 위한 모듈

# play

## 2. 전역 변수 설정
- 게임에서 사용할 주요 변수들을 선언합니다.
- mouse_x, mouse_y는 마우스의 현재 위치를 저장합니다.
- mouse_c는 마우스 클릭 상태를 나타내는 변수입니다.
- cnt는 플레이어가 목적지에 도착한 횟수를 카운트합니다.
- player_speeds, player_stopped, players는 각각 플레이어의 속도, 정지 여부, 생성된 플레이어의 정보를 저장합니다.

In [5]:
# 전역 변수
mouse_x, mouse_y, mouse_c = 0, 0, 0  # 마우스 좌표 및 클릭 상태
cnt = 0  # 플레이어 도착 카운터
player_speeds, player_stopped, players = {}, {}, []  # 플레이어 상태 관리용 딕셔너리

## 마우스 이벤트 처리 함수
- **`mouse_move(e)`**: 마우스의 움직임을 추적하여 `mouse_x`, `mouse_y` 전역 변수에 현재 좌표를 저장하는 함수.
- **`mouse_press(e)`**: 마우스 클릭 이벤트를 처리하여 `mouse_c` 변수를 1로 설정하는 함수.

In [6]:
def mouse_move(e):
    """마우스 움직임 이벤트 처리"""
    global mouse_x, mouse_y
    mouse_x, mouse_y = e.x, e.y

def mouse_press(e):
    """마우스 클릭 이벤트 처리"""
    global mouse_c
    mouse_c = 1

## 윈도우 및 캔버스 설정
- **윈도우 생성 및 설정**: `root` 객체는 tkinter 윈도우를 생성하며, 제목과 크기를 설정합니다.
- **마우스 이벤트 바인딩**: 마우스 움직임과 클릭 이벤트를 각각 `mouse_move`, `mouse_press` 함수와 연결합니다.
- **캔버스 설정**: 1280x720 크기의 캔버스를 생성하고 `root` 윈도우에 추가합니다.

## 초기 페이지 및 전역 변수 설정
- **초기 배경 이미지 설정**: `img` 객체에 초기 배경 이미지를 불러와 중앙에 위치시킵니다.
- **전역 변수 설정**: 현재 페이지 상태 (`index`), 이미지 ID, 이미지 객체 리스트, 이미지 ID 리스트를 초기화합니다.


In [7]:
# 윈도우 및 캔버스 설정
root = tk.Tk()
root.title('노부부와 보호인')
root.resizable(False, False)
root.bind("<Motion>", mouse_move)  # 마우스 움직임 이벤트
root.bind("<ButtonPress>", mouse_press)  # 마우스 클릭 이벤트
root.geometry('1280x720')

canvas = tk.Canvas(root, width=1280, height=720)
canvas.pack()

# 초기 페이지 배경 설정
img = tk.PhotoImage(file='zzzimg/background_pages/start_page.png')
canvas.create_image(640, 360, image=img)

# 전역 변수 설정
index, image_id, img_objects, image_ids = 'start_page', None, [], []



# 팀 정보 전역 변수 추가
# 전역 변수 설정 부분에 추가
teams = [
    {"name": "Team A", "players": ["cat", "dog"], "positions": [0, 0]},
    {"name": "Team B", "players": ["frog", "panda"], "positions": [0, 0]},
    {"name": "Team C", "players": ["pig", "rabbit"], "positions": [0, 0]}
]
current_team_index = 0


TclError: image "pyimage2" doesn't exist

## 아이템 생성 함수
- **함수 설명**: `create_items` 함수는 아이템을 캔버스 위에 랜덤한 위치에 생성하며, 아이템 간의 겹침을 방지합니다.
- **겹침 방지 함수**: `is_overlapping` 함수는 생성된 아이템의 좌표와 비교하여 아이템이 서로 겹치지 않도록 합니다.
- **랜덤 위치 생성**: 각 아이템은 `random.randint`를 사용해 무작위로 생성된 좌표에 배치되며, 위치가 겹치지 않을 때까지 반복합니다.
- **아이템 이미지 로드**: `tk.PhotoImage`를 사용해 이미지를 로드하며, 각 아이템의 정보를 바탕으로 캔버스에 이미지를 추가합니다.
- **리턴 값**: 최종적으로 생성된 아이템 리스트를 반환합니다. 아이템이 성공적으로 생성되지 않을 경우 오류 메시지가 출력됩니다.


In [40]:
def create_items(canvas, item_info, images):
    """아이템을 화면에 생성하는 함수"""
    items = []

    def is_overlapping(x, y, item_size):
        """아이템 간 겹침 방지"""
        for item, _ in items:
            existing_x, existing_y = canvas.coords(item)
            min_dist_x, min_dist_y = item_size[0] + 100, item_size[1] + 100
            if (existing_x - min_dist_x < x < existing_x + min_dist_x and
                existing_y - min_dist_y < y < existing_y + min_dist_y):
                return True
        return False

    for item_type, info in item_info.items():
        item_photo = tk.PhotoImage(file=info['path'])
        images.append(item_photo)
        item_size = (item_photo.width() // 2, item_photo.height() // 2)

        for _ in range(info["count"]):
            while True:
                x, y = random.randint(100, 928), random.randint(50, 670)  # 랜덤 위치 생성
                if not is_overlapping(x, y, item_size):
                    break

            item = canvas.create_image(x, y, image=item_photo)
            if item:
                items.append((item, item_type))
            else:
                print(f"{item_type} 생성 실패")
    
    return items

## 게임 창 생성 함수
- **함수 설명**: `create_game_window` 함수는 게임 시작 시 아이템과 플레이어를 생성하고, 게임 시작 버튼을 추가합니다.
- **아이템 정보 설정**: `item_info` 딕셔너리는 각 아이템의 경로와 생성할 아이템의 개수를 정의합니다.
- **아이템 생성**: `create_items` 함수를 호출하여 캔버스에 아이템을 배치합니다. 생성된 아이템을 `items` 리스트에 저장합니다.
- **플레이어 이미지 설정**: 7개의 플레이어 이미지(고양이, 개, 개구리, 판다, 돼지, 토끼, 다람쥐)를 로드하여 `root.images`에 추가합니다.
- **플레이어 생성**: 각 플레이어는 화면 왼쪽에 고정된 위치에서 생성됩니다. 각 플레이어 이미지를 캔버스에 추가하고 `players` 리스트에 저장합니다.
- **스타트 버튼 추가**: 게임 시작 버튼을 생성하고, 버튼 클릭 시 레이스가 시작되도록 `start_race` 함수를 호출합니다. 버튼은 화면 우측 하단에 배치됩니다.
- **리턴 값**: 생성된 플레이어와 아이템 리스트를 반환합니다.


In [41]:
def create_game_window():
    """게임 시작 시 아이템 및 플레이어 생성"""
    global players

    item_info = {
        "chestnut": {"path": "img/items/chestnut.png", "count": 3},
        "cobweb": {"path": "img/items/cobweb.png", "count": 3},
        "poison_mushroom": {"path": "img/items/poison_mushroom.png", "count": 3},
        "balloon": {"path": "img/items/balloon.png", "count": 3},
        "gold_mushroom": {"path": "img/items/gold_mushroom.png", "count": 3},
        "hole": {"path": "img/items/hole.png", "count": 3},
        "random_box": {"path": "img/items/random_box.png", "count": 3},
    }

    root.images = []  # 이미지 참조를 유지하여 삭제 방지
    items = create_items(canvas, item_info, root.images)

    player_images = [
        tk.PhotoImage(file='img/players/cat.png'),
        tk.PhotoImage(file='img/players/dog.png'),
        tk.PhotoImage(file='img/players/frog.png'),
        tk.PhotoImage(file='img/players/panda.png'),
        tk.PhotoImage(file='img/players/pig.png'),
        tk.PhotoImage(file='img/players/rabbit.png'),
        tk.PhotoImage(file='img/players/squirrel.png'),
    ]

    root.images.extend(player_images)  # 플레이어 이미지 유지

    players = []
    for i in range(7):
        player = canvas.create_image(24, 72 + i * 96, image=player_images[i])
        players.append(player)

    start_button = ttk.Button(root, text="Start", command=lambda: start_race(canvas, players, start_button, items))
    start_button.place(x=1100, y=600)
    return players, items

## 게임 시작 함수
- **함수 설명**: `start_race` 함수는 게임을 시작하고 각 플레이어의 이동을 제어합니다.
- **스타트 버튼 비활성화**: 버튼이 클릭되면 더 이상 클릭할 수 없도록 비활성화합니다.
- **플레이어 이동 시작**: 각 플레이어에 대해 `canvas.after`를 사용하여 지연된 시간(100ms) 후에 `move_player` 함수를 호출합니다. 이때, 람다 함수를 사용하여 현재 플레이어를 매개변수로 전달합니다.
- **아이템과 버튼**: 플레이어는 주어진 아이템을 피하거나 수집하며 이동하게 됩니다. 


In [42]:
def start_race(canvas, players, start_button, items):
    """게임 시작: 플레이어가 이동을 시작"""
    start_button.config(state='disabled')
    for player in players:
        canvas.after(100, lambda p=player: move_player(canvas, p, items, start_button))

## 플레이어 이동 함수
- **함수 설명**: `move_player` 함수는 각 플레이어의 움직임을 제어합니다.
- **플레이어 속도 및 정지 상태 초기화**: `player_speeds`와 `player_stopped` 딕셔너리를 사용하여 각 플레이어의 기본 속도와 정지 상태를 설정합니다.
- **정지 상태 처리**: 만약 플레이어가 정지 상태라면, 100ms 후에 다시 `move_player` 함수를 호출하여 계속 상태를 확인합니다.
- **플레이어 이동**: `dx`와 `dy`를 계산하여 플레이어를 이동시키며, 화면 경계를 넘지 않도록 조정합니다.
- **아이템 충돌 처리**: 모든 아이템과의 충돌을 검사하여, 충돌이 발생할 경우 `handle_item_collision` 함수를 호출하여 적절한 처리를 합니다.
- **완주 체크**: 플레이어가 경주를 완주했는지 확인하고, 그렇지 않다면 계속해서 이동을 반복합니다.


In [43]:
def move_player(canvas, player, items, start_button):
    """플레이어가 움직이는 함수"""
    player_speeds.setdefault(player, 4)  # 기본 속도 4
    player_stopped.setdefault(player, False)  # 기본 정지 상태 False

    if player_stopped[player]:  # 플레이어가 정지 상태일 때
        canvas.after(100, lambda: move_player(canvas, player, items, start_button))
        return

    dx, dy = player_speeds[player], random.choice([-2, 0, 2]) * player_speeds[player]
    canvas.move(player, dx, dy)  # 플레이어 이동

    player_coords = canvas.coords(player)
    if player_coords[1] < 20 or player_coords[1] > 700:  # 화면 밖으로 나가는 것 방지
        canvas.move(player, 0, -dy)

    # 아이템 충돌 처리
    for item, item_type in items[:]:
        if check_collision(player_coords, canvas.coords(item)):
            handle_item_collision(canvas, player, item_type, item, items)

    if not check_finish(canvas, player, start_button):  # 완주 체크
        canvas.after(100, lambda: move_player(canvas, player, items, start_button))


## 아이템 충돌 처리 함수
- **함수 설명**: `handle_item_collision` 함수는 플레이어가 특정 아이템과 충돌했을 때 발생하는 행동을 정의합니다.
- **아이템 종류별 행동**:
  - **"chestnut"**: 플레이어를 왼쪽으로 100픽셀 이동시킵니다.
  - **"cobweb"**: 플레이어를 정지 상태로 만들고, 3초 후에 `resume_player` 함수를 호출하여 다시 움직일 수 있도록 합니다.
  - **"ballon"**: 플레이어를 오른쪽으로 100픽셀 이동시킵니다.
  - **"gold"**: 플레이어의 속도를 5로 증가시킵니다.
  - **"hole"**: 플레이어를 시작 위치로 되돌립니다.
  - **"poison"**: 플레이어의 속도를 3으로 감소시킵니다.
  - **"random"**: `random_action` 함수를 호출하여 랜덤한 행동을 수행합니다.
- **아이템 삭제**: 충돌한 아이템을 캔버스에서 삭제하고, `items` 리스트에서 제거합니다.


In [44]:
def handle_item_collision(canvas, player, item_type, item, items):
    """아이템과 충돌 시 아이템 별 행동 처리"""
    if item_type == "chestnut":
        canvas.move(player, -100, 0)
    elif item_type == "cobweb":
        player_stopped[player] = True
        canvas.after(3000, lambda: resume_player(player))
    elif item_type == "balloon":
        canvas.move(player, 100, 0)
    elif item_type == "gold_mushroom":
        player_speeds[player] = 5
    elif item_type == "hole":
        canvas.move(player, -canvas.coords(player)[0] + 24, 0)
    elif item_type == "poison_mushroom":
        player_speeds[player] = 3
    elif item_type == "random_box":
        random_action(canvas)

    canvas.delete(item)
    items.remove((item, item_type))

## 플레이어 재개 함수
- **함수 설명**: `resume_player` 함수는 정지된 플레이어가 다시 이동할 수 있도록 설정합니다.
- **기능**: 
  - 주어진 플레이어의 `player_stopped` 상태를 `False`로 변경하여, 플레이어가 다시 움직일 수 있게 만듭니다.


In [45]:
def resume_player(player):
    """정지된 플레이어 다시 이동 가능하게 설정"""
    player_stopped[player] = False

## 랜덤 행동 처리 함수
- **함수 설명**: `random_action` 함수는 플레이어들이 수행할 랜덤한 행동을 처리합니다.
- **기능**:
  - **`all_to_start`**: 모든 플레이어를 시작 위치로 되돌립니다.
  - **`shuffle_players`**: 플레이어들의 위치를 무작위로 섞어 재배치합니다.


In [46]:
def random_action(canvas):
    """랜덤 행동 처리"""
    action = random.choice(["all_to_start", "shuffle_players"])

    if action == "all_to_start":
        for player in players:
            canvas.coords(player, 24, canvas.coords(player)[1])
    elif action == "shuffle_players":
        positions = [canvas.coords(player) for player in players]
        random.shuffle(positions)
        for i, player in enumerate(players):
            canvas.coords(player, *positions[i])

## 충돌 확인 함수
- **함수 설명**: `check_collision` 함수는 플레이어와 아이템 간의 충돌 여부를 확인합니다.
- **매개변수**:
  - `player_coords`: 플레이어의 현재 좌표 (x, y).
  - `item_coords`: 아이템의 현재 좌표 (x, y).
- **반환값**: 플레이어와 아이템 간의 충돌 여부를 나타내는 불리언 값.
- **구현 방식**:
  - 플레이어와 아이템의 중심 좌표를 계산합니다.
  - 두 점 사이의 거리를 구하고, 두 객체의 반지름을 합한 값과 비교하여 충돌 여부를 판단합니다.


In [47]:
def check_collision(player_coords, item_coords):
    """플레이어와 아이템 간 충돌 여부 확인"""
    player_center_x, player_center_y = player_coords[0] + 25, player_coords[1] + 25
    item_center_x, item_center_y = item_coords[0] + 17.5, item_coords[1] + 17.5
    distance = ((player_center_x - item_center_x) ** 2 + (player_center_y - item_center_y) ** 2) ** 0.5
    return distance <= (45 / 2) + (30 / 2)

## 완주 확인 함수
- **함수 설명**: `check_finish` 함수는 플레이어가 경주를 완주했는지 확인합니다.
- **매개변수**:
  - `canvas`: 게임의 캔버스 객체.
  - `player`: 현재 확인할 플레이어의 ID.
  - `start_button`: 게임 시작 버튼 객체.
- **전역 변수**: 
  - `cnt`: 현재까지 완주한 플레이어 수를 세기 위한 변수.
  - `index`: 현재 페이지 상태를 나타내는 변수.
  - `image_id`: 현재 표시된 배경 이미지의 ID.
- **구현 방식**:
  - 플레이어의 x좌표가 1008 이상인지 확인합니다.
  - 모든 플레이어가 완주했을 경우, 게임 종료 화면으로 전환하고 배경 이미지를 변경합니다.
  - 완주 여부에 따라 True 또는 False를 반환합니다.


In [48]:
def check_finish(canvas, player, start_button):
    """플레이어가 완주했는지 확인"""
    global cnt, index, image_id
    if canvas.coords(player)[0] >= 1008:
        cnt += 1
        if cnt == 7:  # 모든 플레이어가 완주했을 때
            start_button.destroy()
            img = tk.PhotoImage(file='img/background_pages/end_page1.png')
            img_objects.append(img)
            canvas.delete(image_id)
            image_id = canvas.create_image(640, 360, image=img)
            index = 'end_page1'
            game_main()
            

            start_button.destroy()
            root.after(1000, lambda: start_team_races(canvas))  # 1초 후에 팀별 개인전 시작

        return True
    return False

## 이미지 로드 함수
- **함수 설명**: `load_image` 함수는 지정된 경로에서 이미지를 로드하여 반환합니다.
- **매개변수**:
  - `file_path`: 로드할 이미지 파일의 경로.
- **전역 변수**:
  - `img_objects`: 이미지 객체를 저장하기 위한 리스트. 이미지가 가비지 컬렉션에 의해 삭제되지 않도록 유지합니다.
- **구현 방식**:
  - `tk.PhotoImage`를 사용하여 이미지를 로드하고, 로드된 이미지를 `img_objects` 리스트에 추가합니다.
  - 최종적으로 로드된 이미지를 반환합니다.


In [49]:
def load_image(file_path):
    """이미지 로드 함수"""
    img = tk.PhotoImage(file=file_path)
    img_objects.append(img)
    return img

## 페이지 전환 처리 함수
- **함수 설명**: `switch_page` 함수는 새로운 페이지로의 전환을 처리합니다.
- **매개변수**:
  - `new_page`: 전환할 새로운 페이지의 이름.
  - `image_path`: 새로 표시할 이미지의 파일 경로.
- **전역 변수**:
  - `image_id`: 현재 표시되고 있는 이미지의 ID. 이전 이미지를 삭제하는 데 사용됩니다.
  - `index`: 현재 페이지 상태를 저장하는 변수.
- **구현 방식**:
  - `load_image` 함수를 호출하여 새로운 이미지를 로드합니다.
  - 현재 이미지가 존재할 경우, `canvas.delete`를 사용하여 해당 이미지를 삭제합니다.
  - 새로운 이미지를 캔버스에 생성하고, `image_id`에 저장합니다.
  - `index` 변수를 업데이트하여 현재 페이지 상태를 반영합니다.


In [50]:
def switch_page(new_page, image_path):
    """페이지 전환을 처리하는 함수"""
    global image_id
    img = load_image(image_path)
    if image_id:  # 이전 이미지 삭제
        canvas.delete(image_id)
    image_id = canvas.create_image(640, 360, image=img)  # 새로운 이미지 생성
    global index
    index = new_page  # 페이지 상태 업데이트


## 마우스 클릭 영역 확인 함수
- **함수 설명**: `handle_click` 함수는 마우스 클릭이 특정 영역 내에서 발생했는지 확인합니다.
- **매개변수**:
  - `x1`: 클릭 영역의 왼쪽 경계 x 좌표.
  - `y1`: 클릭 영역의 위쪽 경계 y 좌표.
  - `x2`: 클릭 영역의 오른쪽 경계 x 좌표.
  - `y2`: 클릭 영역의 아래쪽 경계 y 좌표.
- **반환값**: 
  - 클릭이 영역 내에서 발생하면 `True`를 반환하고, 그렇지 않으면 `False`를 반환합니다.
- **구현 방식**:
  - 현재 마우스의 x 좌표(`mouse_x`)와 y 좌표(`mouse_y`)가 주어진 영역의 경계 내에 있는지 비교하여 클릭 여부를 판단합니다.


In [51]:
def handle_click(x1, y1, x2, y2):
    """마우스 클릭 영역을 확인하는 함수"""
    return x1 < mouse_x < x2 and y1 < mouse_y < y2

## 게임 메인 루프 함수
- **함수 설명**: `game_main` 함수는 게임의 다양한 페이지 상태를 관리하고, 마우스 클릭 이벤트에 따라 페이지를 전환합니다.
- **전역 변수**:
  - `mouse_c`: 마우스 클릭 상태.
  - `index`: 현재 페이지 상태를 나타내는 변수.
  - `image_id`: 현재 표시된 이미지의 ID.
  - `image_ids`: 생성된 이미지의 ID 리스트.
  - `cnt`: 완주한 플레이어 수.

- **상태별 처리**:
  - **intro_page**: 
    - 클릭 시 다른 페이지로 전환합니다.
  - **dc_page1** ~ **dc_page4**: 
    - 각 페이지에서 다음 페이지로 이동하거나 이전 페이지로 돌아갈 수 있습니다.
  - **info_page**: 
    - 클릭한 위치에 정보를 표시하고, 특정 클릭 시 페이지 전환.
  - **end_page1** 및 **end_page2**: 
    - 종료 페이지에서 다양한 버튼 클릭에 따라 페이지 전환을 처리합니다.

- **주요 로직**:
  - 클릭 이벤트가 발생하면 해당 영역에 따라 페이지를 전환하고, 마우스 클릭 상태를 초기화합니다.
  - 마지막에, 현재 페이지가 `main_page`가 아닐 경우 100ms 후에 `game_main` 함수를 재호출하여 지속적으로 업데이트합니다.
  - 만약 현재 페이지가 `main_page`라면 `create_game_window` 함수를 호출하여 게임을 시작합니다.


In [52]:
def start_team_races(canvas):
    global current_team_index, index
    print(f"Starting team races. Current team index: {current_team_index}")
    print(f"Teams: {teams}")
    
    if current_team_index < len(teams):
        print(f"Starting race for {teams[current_team_index]['name']}")
        index = 'team_race'
        canvas.delete("all")  # 캔버스 초기화
        try:
            img = load_image('img/background_pages/team_race_background.png')
            canvas.create_image(640, 360, image=img)
            print("Background image loaded successfully")
        except Exception as e:
            print(f"Error loading background image: {e}")
        start_team_race(canvas, teams[current_team_index])
    else:
        print("All team races finished")
        switch_page('end_page1', 'img/background_pages/end_page1.png')

def start_team_race(canvas, team):
    player_objects = []
    for i, player in enumerate(team["players"]):
        player_img = load_image(f'img/players/{player}.png')
        y_pos = 200 + i * 200
        player_obj = canvas.create_image(50, y_pos, image=player_img)
        player_objects.append(player_obj)

    start_button = ttk.Button(root, text=f"Start {team['name']} Race", command=lambda: run_team_race(canvas, team, player_objects))
    start_button.place(x=1100, y=600)

def run_team_race(canvas, team, player_objects):
    def move_players():
        finished = 0
        for player_obj in player_objects:
            if canvas.coords(player_obj)[0] < 1008:
                move = random.randint(1, 5)
                canvas.move(player_obj, move, 0)
            else:
                finished += 1
        
        if finished == len(player_objects):
            root.after(1000, lambda: show_team_results(canvas, team, player_objects))
        else:
            root.after(50, move_players)
    
    move_players()

def show_team_results(canvas, team, player_objects):
    global current_team_index
    result = f"{team['name']} Race Finished!\n\nRankings:\n"
    finished_players = sorted(player_objects, key=lambda p: canvas.coords(p)[0], reverse=True)
    for i, player in enumerate(finished_players):
        result += f"{i+1}. {team['players'][player_objects.index(player)]}\n"
    
    messagebox.showinfo("Team Race Results", result)
    current_team_index += 1
    root.after(1000, lambda: start_team_races(canvas))

    
def show_final_results():
    global index, image_id
    switch_page('end_page1', 'img/background_pages/end_page1.png')

In [53]:
# def game_main():
#     global mouse_c, index, image_id, image_ids, cnt
#     cnt = 0

#     if index == 'start_page':
#         if mouse_c == 1:
#             if handle_click(300, 580, 550, 690):
#                 switch_page('description_1', 'img/background_pages/description_1.png')
#             if handle_click(730, 580, 980, 690):
#                 switch_page('team_input', 'img/background_pages/team_input.png')
#             mouse_c = 0

#     elif index == 'description_1':
#         if mouse_c == 1:
#             if handle_click(1080, 150, 1180, 250):
#                 switch_page('description_2', 'img/background_pages/description_2.png')
#             elif handle_click(100, 150, 200, 250):
#                 switch_page('start_page', 'img/background_pages/start_page.png')
#             mouse_c = 0

#     elif index == 'description_2':
#         if mouse_c == 1:
#             if handle_click(1080, 150, 1180, 250):
#                 switch_page('description_3', 'img/background_pages/description_3.png')
#             elif handle_click(100, 150, 200, 250):
#                 switch_page('description_1', 'img/background_pages/description_1.png')
#             mouse_c = 0

#     elif index == 'description_3':
#         if mouse_c == 1:
#             if handle_click(1080, 150, 1180, 250):
#                 switch_page('description_item', 'img/background_pages/description_item.png')
#             elif handle_click(100, 150, 200, 250):
#                 switch_page('description_2', 'img/background_pages/description_2.png')
#             mouse_c = 0

#     elif index == 'description_item':
#         if mouse_c == 1 and handle_click(540, 600, 740, 700):
#             switch_page('start_page', 'img/background_pages/start_page.png')
#             mouse_c = 0

#     elif index == 'team_input':
#         if mouse_c == 1:
#             img = load_image('img/icons/input2.png')
#             if handle_click(1120, 110, 1200, 155):
#                 image_ids.append(canvas.create_image(635, 205, image=img))
#             elif handle_click(1120, 185, 1200, 230):
#                 image_ids.append(canvas.create_image(635, 280, image=img))
#             elif handle_click(1120, 260, 1200, 305):
#                 image_ids.append(canvas.create_image(635, 355, image=img))
#             elif handle_click(1120, 335, 1200, 380):
#                 image_ids.append(canvas.create_image(635, 430, image=img))
#             elif handle_click(1120, 410, 1200, 455):
#                 image_ids.append(canvas.create_image(635, 505, image=img))
#             elif handle_click(1120, 485, 1200, 530):
#                 image_ids.append(canvas.create_image(635, 580, image=img))
#             elif handle_click(1050, 620, 1210, 700):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#                 for img_id in image_ids:
#                     canvas.delete(img_id)
#                 image_ids = []  # 이미지 ID 리스트 초기화
#             mouse_c = 0

#     elif index == 'end_page1':
#         if mouse_c == 1:
#             if handle_click(30, 400, 330, 550):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(800, 175, 840, 205):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(800, 240, 840, 270):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(800, 305, 840, 335):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(800, 370, 840, 400):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(800, 435, 840, 465):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(800, 500, 840, 530):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(800, 565, 840, 595):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(30, 550, 330, 700):
#                 switch_page('intro_page', 'img/background_pages/intro_page.png')
#             mouse_c = 0

#     elif index == 'end_page2':
#         if mouse_c == 1:
#             if handle_click(30, 550, 330, 700):
#                 switch_page('start_page', 'img/background_pages/intro_page.png')
#             elif handle_click(770, 175, 840, 205):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(770, 240, 840, 270):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(770, 305, 840, 335):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(770, 370, 840, 400):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(770, 435, 840, 465):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(770, 500, 840, 530):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             elif handle_click(770, 565, 840, 595):
#                 switch_page('game_page', 'img/background_pages/game_page.png')
#             mouse_c = 0

    

#     if index != 'game_page':
#         root.after(100, game_main)  # 100ms 후에 다시 호출
#     else:
#         create_game_window()
        
# # 게임 시작
# game_main()

# # Tkinter 메인 루프 실행
# root.mainloop()

def game_main():
    global mouse_c, index, image_id, image_ids, cnt, current_team_index
    cnt = 0

    if index == 'start_page':
        if mouse_c == 1:
            if handle_click(300, 580, 550, 690):
                switch_page('description_1', 'img/background_pages/description_1.png')
            if handle_click(730, 580, 980, 690):
                switch_page('team_input', 'img/background_pages/team_input.png')
            mouse_c = 0

    elif index == 'description_1':
        if mouse_c == 1:
            if handle_click(1080, 150, 1180, 250):
                switch_page('description_2', 'img/background_pages/description_2.png')
            elif handle_click(100, 150, 200, 250):
                switch_page('start_page', 'img/background_pages/start_page.png')
            mouse_c = 0

    elif index == 'description_2':
        if mouse_c == 1:
            if handle_click(1080, 150, 1180, 250):
                switch_page('description_3', 'img/background_pages/description_3.png')
            elif handle_click(100, 150, 200, 250):
                switch_page('description_1', 'img/background_pages/description_1.png')
            mouse_c = 0

    elif index == 'description_3':
        if mouse_c == 1:
            if handle_click(1080, 150, 1180, 250):
                switch_page('description_item', 'img/background_pages/description_item.png')
            elif handle_click(100, 150, 200, 250):
                switch_page('description_2', 'img/background_pages/description_2.png')
            mouse_c = 0

    elif index == 'description_item':
        if mouse_c == 1 and handle_click(540, 600, 740, 700):
            switch_page('start_page', 'img/background_pages/start_page.png')
            mouse_c = 0

    elif index == 'team_input':
        if mouse_c == 1:
            img = load_image('img/icons/input2.png')
            if handle_click(1120, 110, 1200, 155):
                image_ids.append(canvas.create_image(635, 205, image=img))
            elif handle_click(1120, 185, 1200, 230):
                image_ids.append(canvas.create_image(635, 280, image=img))
            elif handle_click(1120, 260, 1200, 305):
                image_ids.append(canvas.create_image(635, 355, image=img))
            elif handle_click(1120, 335, 1200, 380):
                image_ids.append(canvas.create_image(635, 430, image=img))
            elif handle_click(1120, 410, 1200, 455):
                image_ids.append(canvas.create_image(635, 505, image=img))
            elif handle_click(1120, 485, 1200, 530):
                image_ids.append(canvas.create_image(635, 580, image=img))
            elif handle_click(1050, 620, 1210, 700):
                switch_page('game_page', 'img/background_pages/game_page.png')
                for img_id in image_ids:
                    canvas.delete(img_id)
                image_ids = []  # 이미지 ID 리스트 초기화
            mouse_c = 0

    elif index == 'end_page1':
        if mouse_c == 1:
            if handle_click(30, 400, 330, 550):
                current_team_index = 0  # 팀 인덱스 초기화
                start_team_races(canvas)  # 팀별 개인전 시작
            elif handle_click(800, 175, 840, 205):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(800, 240, 840, 270):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(800, 305, 840, 335):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(800, 370, 840, 400):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(800, 435, 840, 465):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(800, 500, 840, 530):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(800, 565, 840, 595):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(30, 550, 330, 700):
                switch_page('intro_page', 'img/background_pages/intro_page.png')
            mouse_c = 0

    elif index == 'end_page2':
        if mouse_c == 1:
            if handle_click(30, 550, 330, 700):
                switch_page('start_page', 'img/background_pages/intro_page.png')
            elif handle_click(770, 175, 840, 205):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(770, 240, 840, 270):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(770, 305, 840, 335):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(770, 370, 840, 400):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(770, 435, 840, 465):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(770, 500, 840, 530):
                switch_page('game_page', 'img/background_pages/game_page.png')
            elif handle_click(770, 565, 840, 595):
                switch_page('game_page', 'img/background_pages/game_page.png')
            mouse_c = 0

    elif index == 'team_race':
        # 팀별 개인전 진행 중 필요한 로직 추가
        pass

    if index not in ['game_page', 'team_race']:
        root.after(100, game_main)  # 100ms 후에 다시 호출
    elif index == 'game_page':
        create_game_window()

# 게임 시작
game_main()

# Tkinter 메인 루프 실행
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\anaconda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\hyejo\AppData\Local\Temp\ipykernel_32632\1163872183.py", line 17, in <lambda>
    root.after(1000, lambda: start_team_races(canvas))  # 1초 후에 팀별 개인전 시작
                             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hyejo\AppData\Local\Temp\ipykernel_32632\509445470.py", line 6, in start_team_races
    img = load_image('img/background_pages/team_race_background.png')
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hyejo\AppData\Local\Temp\ipykernel_32632\3952474727.py", line 3, in load_image
    img = tk.PhotoImage(file=file_path)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\tkinter\__init__.py", line 4129, in __init__
    Image.__init__(